#### Preparar Ambiente

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
!{sys.executable} -m pip install pytz

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


#### Conectar con pyathena

In [3]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [4]:
from datetime import datetime, timedelta
import numpy as np

#### Crear tablas por pais

In [5]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [6]:
pais = 'BR'

In [7]:
niv_adm = '4'
id_adm = 'id_adm'

#### Crear matriz origen-destino final

In [8]:
start_time = time.time()
tabla_matrizOD_adms_pais = f'historico_mod_adm{niv_adm}_{pais}'
tabla_matrizOD_adms_pais_final = f'historico_mod_adm{niv_adm}_{pais}_final'
query = f'''
    CREATE TABLE IF NOT EXISTS {tabla_matrizOD_adms_pais_final} 
    WITH ( external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-public/mod/{pais}/adm{niv_adm}/{tabla_matrizOD_adms_pais_final}', 
                        format = 'PARQUET', 
                        parquet_compression = 'SNAPPY') AS
    SELECT *
    FROM {tabla_matrizOD_adms_pais}
    '''
cursor.execute(query)
print((time.time()-start_time)/60)
print(f'Generacion de tabla {tabla_matrizOD_adms_pais_final}: terminado')

0.07324079275131226
Generacion de tabla historico_mod_adm4_BR_final: terminado


In [9]:
tabla_matrizOD_adms_pais_final = f'historico_mod_adm{niv_adm}_{pais}_final'
query = f'''
    SELECT count(*) as n,
    COUNT(distinct (id_adm)) as n_id_adm
    FROM {tabla_matrizOD_adms_pais_final}
    '''
cursor.execute(query).as_pandas()

,n,n_id_adm
0,8522146,5565


In [10]:
tabla_matrizOD_adms_pais_final = f'historico_mod_adm{niv_adm}_{pais}_final'
query = f'''
    SELECT *
    FROM {tabla_matrizOD_adms_pais_final}
    LIMIT 10
    '''
df = cursor.execute(query).as_pandas()

In [11]:
df

,year,month,day,id_adm,id_admh,n
0,2020,3,19,2604809.0,2602308.0,2
1,2020,3,19,4314209.0,4301305.0,2
2,2020,3,19,3552551.0,3546603.0,2
3,2020,3,19,3533403.0,3510609.0,2
4,2020,3,19,3152501.0,3549904.0,2
5,2020,3,19,4318440.0,4309258.0,2
6,2020,3,19,3143302.0,3148004.0,2
7,2020,3,19,2700201.0,2704302.0,2
8,2020,3,19,3550308.0,3164407.0,2
9,2020,3,19,2410801.0,2409407.0,2
